# Leontief model definition
Input-output analysis is a form of macroeconomic analysis based on the interdependencies between different economic sectors or industries.
As you know the ecconomy divides into sectors where each sector produces goods and services not only for itself but also for other sectors so each sector has an effect on others.
This model helps us to find out how much goods do we need to produce in order to meet the outside and intermidiate demands.
The leontief model is branched into two submodels : closed and open.
- open model : some production is consumed by industries, and the rest is consumed by external bodies.
- closed model : all production is consumed by industries.

the equation for the open model is x = Cx + d, where d represents he demand matrix. In a closed economy, the equation is x = Cx, which means the total input equals the total output.

# Problem description

Suppose an open economy consists of Coal, Electric, and Steel sectors, and the output of each sector is distributed among the various sectors as shown in the table below,
where the entries in a column represent the fractional parts of a sectors total output.

<img src="table.jpg" alt = "simple economy" width="400"/>

suppose that the open sector has a demand for $7900 worth of coal, $3950 worth of Electric and $1975 worth of steel.
- a) Can the economy meet the demand?
- b) If so, find a production vector x that will meet it exactly.


### 0. Libraries

In [ ]:
import numpy as np

## 1. Deriving consumption matrix
 Consumption matrix C : this matrix's columns are the inputs required for each output.
 


In [ ]:
Consumption = np.array([[0.5,0.1,0.1]
              ,[0.2,0.5,0.3]
              ,[0.1,0.3,0.4]])

## 2. Finding the echelon form of the augmented matrix

<p>
  In this part, we need to use the function defined in the next section to find the echelon form of the augmented matrix [I - C | d]. The column vector <i>d</i> is called the outside demand vector. Since the product-producing sectors consume some of their own output, the monetary value of their output must cover both their own needs and the outside demand.
</p>

<p>
  The column vector <i>x</i>, which contains the monetary values as successive components, is called the production vector for the economy. By multiplying <i>x</i> by the consumption matrix <i>C</i>, we obtain <i>Cx</i>, the portion of the production vector that will be consumed by the productive sectors. This vector <i>Cx</i> is referred to as the intermediate demand vector for the economy.
</p>

<p>
  Once the intermediate demand is met, the portion of the production that remains to satisfy the outside demand is <i>x - Cx</i>. Therefore, <i>x</i> must satisfy the equation:
</p>

<p style="text-align: center; font-size: 24px;">
  x - Cx = d
</p>

<p>
  This equation can be rewritten in a more convenient form:
</p>

<p style="text-align: center; font-size: 24px;">
  (I - C)x = d
</p>

<p>
  The matrix <i>I - C</i> is called the Leontief matrix, and the equation <i>(I - C)x = d</i> is known as the Leontief equation.
</p>


### 2.1. Row Echelon Form
concat the I-C matrix and d vector and give the augmented matrix to the function.

In [ ]:
def row_echelon(A):
    rows, cols = A.shape
    current_row = 0

    for col in range(cols):
        # Check for a pivot in the current column
        for r in range(current_row, rows):
            if A[r, col] != 0:
                # Swap rows if we find a non-zero pivot
                A[[current_row, r]] = A[[r, current_row]]
                break
        else:  # No pivot found in this column (column is all-zero), move to the next column
            continue

        # Make entries below the pivot 0 by row operations
        for r in range(current_row + 1, rows):
            if A[r, col] != 0:
                A[r] -= A[r, col] / A[current_row, col] * A[current_row]  # Row operation to make the entry 0

        current_row += 1  # Move to the next row for the next pivot

    return A

   

In [ ]:
#e.g.
C = np.array([[0.5,0.1,0.1]
              ,[0.2,0.5,0.3]
              ,[0.1,0.3,0.4]])
d = np.array([[7900]
              ,[3950]
              ,[1975]])
A = np.array([[0.5,0.9,0.9,7900]
              ,[0.8,0.5,0.7,3950]
              ,[0.9,0.7,0.6,1975]])
print(row_echelon(A))
"""Notice that any true row echelon form is acceptable.Row echelon form (REF) of a matrix isn't unique"""

### 2.2. Reduce Row Echelon Form

In [ ]:
def reduce(A):
    rows, cols = A.shape
    for r in range(rows - 1, -1, -1):  # iterate from last row to first one
        for c in range(cols):
            if A[r, c] != 0:  # pivot found
                A[r] /= A[r, c]
                for row in range(r - 1, -1, -1):  # make the entries above the pivot 0
                    if A[row, c] != 0:
                        A[row] -= A[r] * A[row, c]
                break  # move to the row above
    return A

In [ ]:
# e.g.
B = np.array([[1,-0.2,-0.2,15800]
                ,[0,23,-17,355500]
                ,[0,0,1,24750]])
print(reduce(B))
"""Notice that row reduced echelon form (RREF) of a matrix is unique"""


## 3. The solution
In this section we find the production vector x that covers the intermediate and outside economical demand

### 3.1. Existence of the Solution
Based on the reduced form of the augmented matrix and prior to finding the answer, make sure the solution exists!

In [ ]:
def is_consistent(A):
    row, col = A.shape

    # iterate over the rows from bottom
    for r in range(row - 1, -1, -1):
        # Check if the row contains only zeros in the coefficient part but non-zero in the augmented part
        if not all(A[r, :-1] == 0):
            return True
        if A[r, -1] != 0:
            return False

    return True  # No inconsistencies found


In [ ]:
#e.g.
print(is_consistent(B))

### 3.2. Finding a Solution
If the system has a solution, find and return it.
If the system has infinite solutions, return only one solution. (If there are some free variables assume them as '0')

In [ ]:
def solve(A):
    row_reduced_A = reduce(row_echelon(A))
    if not is_consistent(row_reduced_A):
        return "this system has no answer"
    
    row, col = A.shape
    num_variables = col - 1  # matrix is augmented
    solution = [0] * num_variables  # default value for free variables is 0
    for r in range(row):
        pivot_index = -1  # find the index of pivot in each row
        for c in range(col):
            if A[r, c] != 0:
                pivot_index = c
                break
        
        if pivot_index == -1:  # if current row doesn't have a pivot, move to the next column
            continue
        
        solution[pivot_index] = A[r, -1]
    solution_text = []
    for index, s in enumerate(solution):
        solution_text.append(f'x{index + 1} = {s}')
    return ' , '.join(solution_text)
    
    

In [ ]:
#e.g.
B = np.array([[ 1, 0, 0, 27500],
              [ 0, 1, 0, 33750],
              [ 0, 0, 1, 24750]], dtype=float)
print(solve(B))

A = np.array([[1,2,-1,0],
              [3,6,0,4],
              [2,4,1,3]], dtype=float)
print(solve(A))

D = np.array([[1,2,3],
              [4,5,6],
              [7,8,9]], dtype=float)
print(solve(D))


E = np.array([[1,2,3,4],
              [0,1,2,5]], dtype=float)
print(solve(E))